In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import automated web-scraping modules
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# import time library for waiting
import time

# import numpy and pandas library for numerical data manipulation and exploratory analysis
import pandas as pd
import numpy as np

# import re for manipulating regular expressions
import re

# import matplotlib and seaborn for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

<b>WEB-SCRAPING</b>

In [4]:
# Install Chrome Driver Manager
driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.58M/6.58M [00:32<00:00, 210kB/s]


In [5]:
# Communicate your user-agent
opts = Options()
opts.add_argument
("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36")
driver = webdriver.Chrome(chrome_options=opts)

C:\Users\assaa\AppData\Local\Temp\ipykernel_26992\1471295826.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=opts)


In [6]:
# Open the first page that we need data from
driver.get("https://www.superimmo.com/achat/ile-de-france/p/0")

# Take a break
time.sleep(2)

# Find the button of the pop-up window on the web page
button = driver.find_element(By.XPATH,"//*[@id='tarteaucitronPersonalize2']")

# Take a break
time.sleep(2)

# Click the button
button.click()

# Take a break
time.sleep(2)

In [7]:
# Loop over the real estate ads pages to scrape all the ad data necessary

# Create the list that will hold all the ads scraped at the end of the process
all_data_dict = []

# Choose the number of pages to loop over - here we chose 6 pages
for n in range(1,6):
    
    # Choose the number of ads per page to loop over - here we chose 10 ads
    for m in range(1,11):
        
        # Open each page with the below command 
        driver.get("https://www.superimmo.com/achat/ile-de-france/p/{}".format(n))
        
        # Take a break and stay ethical 
        time.sleep(0.5)
        
        # Create a dictonary for each page that includes all the features to scrape for each ad
        data_dict = {}

        # Scrape the property price of each ad
        property_price = driver.find_element(by=By.XPATH,value=
                                             "//*[@id='pjax-container']/section/article[{}]/section/div[3]/p/a/b[1]".format(m))

        # Format the property price tightly
        data_dict['price EUR'] = property_price.text
        # Create conditional choices to avoid breaking the process flow
        if data_dict['price EUR'] == '':
            data_dict['price EUR'] = data_dict['price EUR']
        elif isinstance(data_dict['price EUR'] , str) == True:
            data_dict['price EUR'] = data_dict['price EUR']
        else:
            data_dict['price EUR'] = int(re.sub(r'[^0-9]', '', data_dict['price EUR']))
        
        # Scrape the property type, its size, its number of cells and its number of rooms
        type_size_cells_rooms = driver.find_element(by=By.XPATH,value=
                                                    "//*[@id='pjax-container']/section/article[{}]/section/div[3]/p/a/b[2]".format(m))

        # Format the property type tightly
        data_dict['type'] = type_size_cells_rooms.text.split(" • ")
        data_dict['type'] = data_dict['type'][0]

        # Format the property size tightly
        data_dict['size m²'] = type_size_cells_rooms.text.split(" • ")
        
        # Create conditional choices to avoid breaking the process flow
        if " m²" in data_dict['size m²'][1]:
            data_dict['size m²'] = data_dict['size m²'][1]
            data_dict['size m²'] = data_dict['size m²'].replace(" m²", "")
            if "," in data_dict['size m²']:
                data_dict['size m²'] = float(data_dict['size m²'].replace(" m²", "").replace(",","."))
            else:
                data_dict['size m²'] = int(data_dict['size m²'].replace(" ", ""))
        else:
            data_dict['size m²'] == None

        # Format the number of cells of a property tightly
        data_dict['cells'] = type_size_cells_rooms.text
        
        # Create conditional choices to avoid breaking the process flow
        if "pièces" in data_dict['cells']:
            sindex = re.search(r"pièces",data_dict['cells']).start()
            data_dict['cells'] = data_dict['cells'][re.search(r"pièces",data_dict['cells']).start()-2:sindex+len(r"pièces")]
            data_dict['cells'] = int(data_dict['cells'].replace(" pièces", ""))
        elif "pièce" in data_dict['cells']:
            sindex = re.search(r"pièce",data_dict['cells']).start()
            data_dict['cells'] = data_dict['cells'][re.search(r"pièce",data_dict['cells']).start()-2:sindex+len(r"pièce")]
            data_dict['cells'] = int(data_dict['cells'].replace(" pièce", ""))
        else:
            data_dict['cells'] = None
            
        # Format the number of rooms of a property tightly
        data_dict['rooms'] = type_size_cells_rooms.text
        
        # Create conditional choices to avoid breaking the process flow
        if "chambres" in data_dict["rooms"]:
            sindex = re.search(r"chambres",data_dict["rooms"]).start()
            data_dict["rooms"] = data_dict["rooms"][re.search(r"chambres",data_dict["rooms"]).start()-2:sindex+len(r"chambres")]
            data_dict["rooms"] = int(data_dict["rooms"].replace(" chambres", ""))
        elif "chambre" in data_dict["rooms"]:
            sindex = re.search(r"chambre",data_dict["rooms"]).start()
            data_dict["rooms"] = data_dict["rooms"][re.search(r"chambre",data_dict["rooms"]).start()-2:sindex+len(r"chambre")]
            data_dict["rooms"] = int(data_dict["rooms"].replace(" chambre", ""))
        else:
            data_dict['rooms'] = None
        
        # Scrape the city and postal code
        city_postcode = driver.find_element(by=By.XPATH,value=
                                            "//*[@id='pjax-container']/section/article[{}]/section/div[3]/b".format(m))

        # Format the city tightly
        data_dict['city']= city_postcode.text
        data_dict['city']= re.sub('[0-9]','', data_dict['city'])
        data_dict['city']= re.sub(r'\W+', '', data_dict['city'])
        
        # Create conditional choices to avoid breaking the process flow
        if data_dict['city'] == "Parisème":
            sindex = re.search(r"Paris",data_dict["city"]).start()
            data_dict['city'] = data_dict['city'][re.search(r"Paris",data_dict['city']).start():sindex+len(r"Paris")]
        else:
            data_dict['city']= data_dict['city']

        # Format the postal code tightly
        data_dict['postal_code']= city_postcode.text
        data_dict['postal_code']= re.sub('[^0-9\,]','', data_dict['postal_code'])
        
        # Create conditional choices to avoid breaking the process flow
        if len(data_dict['postal_code']) == 7:
            data_dict['postal_code'] = data_dict['postal_code'][2:]
        elif len(data_dict['postal_code']) == 6:
            data_dict['postal_code'] = data_dict['postal_code'][1:]
        else:
            data_dict['postal_code'] = data_dict['postal_code']
            
        # Take a break and stay ethical 
        time.sleep(0.5)
        
        # Append every 10 real estate ads scraped to the external list
        all_data_dict.append(data_dict)
        
        # Create a dataframe out of the full list
        df = pd.DataFrame.from_dict(all_data_dict)

In [8]:
# Look at the first 5 rows of the dataframe
df.head(5)

,price EUR,type,size m²,cells,rooms,city,postal_code
0,315 000 €,Maison,135.00,5,4.0,AuthonlaPlaine,91410
1,548 000 €,Appartement,87.00,4,3.0,AsnièressurSeine,92600
2,155 000 €,Appartement,42.09,2,1.0,LesPavillonssousBois,93320
3,175 000 €,Appartement,93.00,5,4.0,Aubergenville,78410
4,435 000 €,Appartement,66.00,3,2.0,Montreuil,93100


<b>EXPLORATORY DATA ANALYSIS</b>

In [ ]:
# Check the shape of the dataset
print("The dataset is composed of", df.shape[0], "rows and", df.shape[1], "columns")

In [ ]:
# Check the information of the dataset
df.info()

In [ ]:
# Calculate the percentage of missing values per feature column with the data type of each column
percent_missing = df.isnull().mean()
missing_value_df = pd.DataFrame(percent_missing).reset_index() # convert to DataFrame
missing_value_df = missing_value_df.rename(columns = {"index" : "feature", 0 : "percent_missing"}) # rename columns
missing_value_df = missing_value_df.sort_values(by = ['percent_missing'], ascending = False) # sort the values
data_types_df = pd.DataFrame(df.dtypes).reset_index().rename(columns = {"index" : "feature", 0 : "data_type"}) # rename columns
missing_value_df = missing_value_df.merge(data_types_df, on = "feature") # join the dataframe with datatype
missing_value_df.percent_missing = round(missing_value_df.percent_missing*100, 2) # format the percent_missing
missing_value_df

In [ ]:
# Change some columns types to be able to manipulate the data
df["size m²"] = pd.to_numeric(df["size m²"])

In [ ]:
# Discover the count of different values within each categorical column 
categorical_columns = list(df.select_dtypes(['object']).columns)
cat_df = []
for c in categorical_columns:
    cat_df.append({"categorical_feature": c, "number_categories": len(df[c].value_counts(dropna = False))})
pd.DataFrame(cat_df).sort_values(by = "number_categories", ascending = False)

In [ ]:
# Fill null values of rooms column with "0"
df["rooms"] = df["rooms"].fillna(0)

# Fill null values of cells column with "1"
df["cells"] = df["cells"].fillna(1)

# Fill null values of size column with the average size of all apartments 
df["size m²"] = df["size m²"].fillna(df["size m²"].mean())

In [ ]:
# Calculate the percentage of missing values one last time to make sure there are no missing data anymore
percent_missing = df.isnull().mean()
missing_value_df = pd.DataFrame(percent_missing).reset_index() # convert to DataFrame
missing_value_df = missing_value_df.rename(columns = {"index" : "feature", 0 : "percent_missing"}) # rename columns
missing_value_df = missing_value_df.sort_values(by = ['percent_missing'], ascending = False) # sort the values
data_types_df = pd.DataFrame(df.dtypes).reset_index().rename(columns = {"index" : "feature", 0 : "data_type"}) # rename columns
missing_value_df = missing_value_df.merge(data_types_df, on = "feature") # join the dataframe with datatype
missing_value_df.percent_missing = round(missing_value_df.percent_missing*100, 2) # format the percent_missing
missing_value_df

In [ ]:
# Check summary statistics of the numerical features of the dataset
round(df.describe())

In [ ]:
# Identify correlations in data
corr = df.corr()
corr

In [ ]:
# Identify patterns to look for correlations between features
correlation_threshold = 0.3
corr_matrix = df.corr().abs() # calculate the correlation matrix with 
high_corr_var = np.where(corr_matrix >= correlation_threshold) # identify variables that have correlations above defined threshold
high_corr_var = [(corr_matrix.index[x], corr_matrix.columns[y], round(corr_matrix.iloc[x, y], 2))
                     for x, y in zip(*high_corr_var) if x != y and x < y] # identify pairs of highly correlated variables
if high_corr_var != []:
    high_corr_var_df = pd.DataFrame(high_corr_var).rename(columns = {0: 'corr_feature',
                                                                 1: 'drop_feature',
                                                                 2: 'corrrelation_values'})
    high_corr_var_df = high_corr_var_df.sort_values(by = 'corrrelation_values', ascending = False)
else:
    high_corr_var_df = print("there are no pairs of correlations with that threshold")
high_corr_var_df

In [ ]:
# Identify features with low variance
std_df = pd.DataFrame(df.std()).rename(columns = {0: 'standard_deviation'})
low_var_features = list(std_df[std_df['standard_deviation'] < 0.1].index)
print("number of low variance features:", len(low_var_features))
print("low variance features:", low_var_features)
low_var_features

# Below the threshold of 10%, there are no low-variance features in the data

In [ ]:
# Check the average count of real estate ads scraped per city
print("Average count of real estate ads per city is:", round(df.groupby('city')['city'].count().mean(),1), "ads")

In [ ]:
# Check the number of cities scraped
print("The number of cities scraped is:", df["city"].nunique(), "cities")

In [ ]:
# Check the percentage representation of top 10 cities in terms of number of properties scraped
round(df["city"].value_counts(normalize = True)*100, 1).head(10)

<b>FEATURE ENGINEERING</b>

In [ ]:
# Create a copy dataframe of df to avoid losing any progress
df_eng = df.copy()

In [ ]:
# Check count of cell values
df_eng["cells"].value_counts()

In [ ]:
# Replace 0 cell with 1 cell
df_eng["cells"] = df_eng["cells"].replace(0.0, 1.0)
df_eng["cells"].value_counts()

In [ ]:
# Create a price per sqm column
df_eng["price/m²"] = round(df_eng["price EUR"] / df_eng["size m²"])

In [ ]:
# Drop price EUR and size m² columns
df_eng = df_eng.drop(columns = ["price EUR", "size m²"])

In [ ]:
# Count unique values of postal codes
df_eng["postal_code"].nunique()

In [ ]:
# Create a district column to decrease the number of postal codes
df_eng["district_code"] = df_eng["postal_code"].str[:2]
df_eng["district_code"] = df_eng["district_code"].replace("", None)
df_eng["district_code"] = df_eng["district_code"].str[:2]

In [ ]:
# Drop the city and postal code columns
df_eng = df_eng.drop(columns = ["city", "postal_code"])

In [ ]:
# Count district code values
df_eng["district_code"].value_counts()

In [ ]:
# Detect outliers in price/m²
def find_outliers_IQR(df_eng):
    q1=df_eng.quantile(0.25)
    q3=df_eng.quantile(0.75)
    IQR=q3-q1
    outliers = df_eng[((df_eng<(q1-1.5*IQR)) | (df_eng>(q3+1.5*IQR)))]
    return outliers
price_per_sqm_outliers = find_outliers_IQR(df_eng["price/m²"])

In [ ]:
# Exclude outliers by limiting the visualized values
y = df_eng["price/m²"][df_eng["price/m²"] < 50000]

# Choose the figure size
plt.figure(figsize=(10,5))

# Choose the grid style
sns.set_style("whitegrid")

# Plot the the price per square meter variations per district code
box_plot = sns.boxplot(data=df_eng, x=df_eng["district_code"].sort_values(ascending=True), y=y)

# Display the plot
plt.show()

In [ ]:
# Check unique types of properties
df_eng["type"].value_counts()

In [ ]:
# Replace "Appartement neuf" with "Appartement" because there is no need for this small complexity
df_eng["type"] = df_eng["type"].replace("Appartement neuf", "Appartement")

In [ ]:
# Drop null values from the dataset
df_eng = df_eng.dropna()

# Re-check the shape of the dataset
print("The dataset is now composed of", df_eng.shape[0], "rows and", df_eng.shape[1], "columns")

In [ ]:
# One-hot encode the categorical variables: type and district_code
for c in df_eng[["type", "district_code"]]:
    dummies = pd.get_dummies(df_eng[c], prefix=c)
    df_eng = pd.concat([df_eng, dummies], axis=1)
    df_eng.drop(c, axis = 1, inplace = True)

In [ ]:
# Show the first 10 rows of the final dataset for machine learning processing
df_eng.head(10)

<b>MACHINE LEARNING MODEL</b>

GRADIENT BOOSTING REGRESSOR

In [ ]:
# Import Sci-kit Learn modules for processing, fitting and evaluation of the Gradient Boosting Regressor model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import learning_curve
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# Transform the data points with Principal Component Analysis 
pca = PCA()
pca = pca.fit(df_eng)
dataPCA = pca.transform(df_eng)
dataPCA = pd.DataFrame(dataPCA)

# Rename the resulting columns of the transformation to their original names before the transformation 
dataPCA = dataPCA.rename(columns = {0:"cells", 1:"rooms", 2:"price/m²", 3:"type_Appartement", 4:"type_Maison", 
                                    5:"district_code_75", 6:"district_code_77", 7:"district_code_78", 8:"district_code_91", 
                                    9:"district_code_92", 10:"district_code_93", 11:"district_code_94", 12:"district_code_95"})

# Take a look on the transformed dataset
dataPCA.head(5)

In [ ]:
# Identify X as the features and y as the variable to predict
X = dataPCA.drop(columns=["price/m²"])
y = dataPCA["price/m²"]

In [ ]:
# Split the X and y into train and test sets
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.3, random_state =42)

In [ ]:
# Instantiate the Gradient Boosting Regressor algorithm
gbr = GradientBoostingRegressor()

In [ ]:
# Fit the model on the train set
gbr.fit(X_train, y_train)

In [ ]:
# Predict the variable
y_pred = gbr.predict(X_test)

In [ ]:
# Evaluate the model by comparing the r_squared score of the train set to the one of the test set
print(r2_score(y_train,gbr.predict(X_train))*100)
print(r2_score(y_test, y_pred)*100)

In [ ]:
# Evaluate the model by checking its mean squared error
mean_squared_error(y_test, y_pred)

In [ ]:
# Plot the learning curve of the model by choosing different metrics for evaluation
train_sizes, train_scores, test_scores = learning_curve(gbr, X, y, cv=10, scoring='r2', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Identify the mean and the standard deviation of the training set 
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Identify the mean and the standard deviation of the test set 
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
# Choose the figure size
plt.subplots(1, figsize=(5,5))

# Plot the trained data sizes on the x-axis and both the training mean and test mean on the y-axis
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Fill the sides of the curves with mean + or - the standard deviation of the dataset for both train and test sets
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Choose a title for the plot
plt.title("Learning Curve")

# Choose labels and legend for the plot
plt.xlabel("Training Set Size"), plt.ylabel("R2 Score"), plt.legend(loc="best")

# Specify the layout style
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Define a feature importance function 
def feature_importance_plot(model, X_train, n):
    """Plots feature importance - this only works for Decision Tree based Models"""
    plt.figure(figsize=(8, 5)) # set figure size
    feat_importances = pd.Series(model.feature_importances_,
                                 index = X_train.columns)
    feat_importances.nlargest(n).plot(kind = 'bar')
    plt.title(f"Top {n} Features")
    plt.show()

In [ ]:
# Plot the feature importance of the Gradient Boosting Regressor model
feature_importance_plot(gbr, X_train, 10)

RANDOM FOREST REGRESSOR

In [ ]:
# Import Random Forest Regressor model
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Transform the data points with Principal Component Analysis 
pca = PCA()
pca = pca.fit(df_eng)
dataPCA = pca.transform(df_eng)
dataPCA = pd.DataFrame(dataPCA)

# Rename the resulting columns of the transformation to their original names before the transformation 
dataPCA = dataPCA.rename(columns = {0:"cells", 1:"rooms", 2:"price/m²", 3:"type_Appartement", 4:"type_Maison", 
                                    5:"district_code_75", 6:"district_code_77", 7:"district_code_78", 8:"district_code_91", 
                                    9:"district_code_92", 10:"district_code_93", 11:"district_code_94", 12:"district_code_95"})

In [ ]:
# Identify X as the features and y as the variable to predict
X = dataPCA.drop(columns=["price/m²"])
y = dataPCA["price/m²"]

In [ ]:
# Split the X and y into train and test sets
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.3, random_state =42)

In [ ]:
# Instantiate the Random Forest Regressor algorithm
rf = RandomForestRegressor()

In [ ]:
# Fit the model on the train set
rf.fit(X_train, y_train)

In [ ]:
# Predict the variable
y_pred = rf.predict(X_test)

In [ ]:
# Evaluate the model by comparing the r_squared score of the train set to the one of the test set
print(r2_score(y_train,rf.predict(X_train))*100)
print(r2_score(y_test, y_pred)*100)

In [ ]:
# Evaluate the model by checking its mean squared error
mean_squared_error(y_test, y_pred)

In [ ]:
# Plot the learning curve of the model by choosing different metrics for evaluation
train_sizes, train_scores, test_scores = learning_curve(rf, X, y, cv=10, scoring='r2', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Identify the mean and the standard deviation of the training set 
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Identify the mean and the standard deviation of the test set 
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
# Choose the figure size
plt.subplots(1, figsize=(5,5))

# Plot the trained data sizes on the x-axis and both the training mean and test mean on the y-axis
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Fill the sides of the curves with mean + or - the standard deviation of the dataset for both train and test sets
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Choose a title for the plot
plt.title("Learning Curve")

# Choose labels and legend for the plot
plt.xlabel("Training Set Size"), plt.ylabel("R2 Score"), plt.legend(loc="best")

# Specify the layout style
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plot the feature importance of the Random Forest Regressor model
feature_importance_plot(rf, X_train, 10)

XGBOOST REGRESSOR

In [ ]:
# Import the XGBoost Regressor model
import xgboost
from xgboost import XGBRegressor

In [ ]:
# Transform the data points with Principal Component Analysis 
pca = PCA()
pca = pca.fit(df_eng)
dataPCA = pca.transform(df_eng)
dataPCA = pd.DataFrame(dataPCA)
dataPCA = dataPCA.rename(columns = {0:"cells", 1:"rooms", 2:"price/m²", 3:"type_Appartement", 4:"type_Maison", 
                                    5:"district_code_75", 6:"district_code_77", 7:"district_code_78", 8:"district_code_91", 
                                    9:"district_code_92", 10:"district_code_93", 11:"district_code_94", 12:"district_code_95"})

In [ ]:
# Identify X as the features and y as the variable to predict
X = dataPCA.drop(columns=["price/m²"])
y = dataPCA["price/m²"]

In [ ]:
# Split the X and y into train and test sets
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.3, random_state =42)

In [ ]:
# Instantiate the XGBoost Regressor algorithm
xgb = XGBRegressor()

In [ ]:
# Fit the model on the train set
xgb.fit(X_train, y_train)

In [ ]:
# Predict the variable
y_pred = xgb.predict(X_test)

In [ ]:
# Evaluate the model by comparing the r_squared score of the train set to the one of the test set
print(r2_score(y_train,xgb.predict(X_train))*100)
print(r2_score(y_test, y_pred)*100)

In [ ]:
# Evaluate the model by checking its mean squared error
mean_squared_error(y_test, y_pred)

In [ ]:
# Plot the learning curve of the model by choosing different metrics for evaluation
train_sizes, train_scores, test_scores = learning_curve(xgb, X, y, cv=10, scoring='r2', n_jobs=-1, train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Identify the mean and the standard deviation of the training set
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Identify the mean and the standard deviation of the test set 
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
# Choose the figure size
plt.subplots(1, figsize=(5,5))

# Plot the trained data sizes on the x-axis and both the training mean and test mean on the y-axis
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Fill the sides of the curves with mean + or - the standard deviation of the dataset for both train and test sets
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Choose a title for the plot
plt.title("Learning Curve")

# Choose labels and legend for the plot
plt.xlabel("Training Set Size"), plt.ylabel("R2 Score"), plt.legend(loc="best")

# Specify the layout style
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plot the feature importance of the XGBoost Regressor model
feature_importance_plot(xgb, X_train, 10)